In [56]:
import torch
import numpy as np 
from datautil import *
from math import log2
train_set, test_set = read_data_as_np(None)
NUM_FEATURES = 3501
DATASET_SIZE = len(train_set)
feature_occ = [0] * NUM_FEATURES
num_p_s = [0] * NUM_FEATURES
num_n_s = [0] * NUM_FEATURES
num_p = 0
num_n = 0

In [57]:
for i in range(DATASET_SIZE):
    fp, tox = train_set[i]
    if tox == 1: num_p += 1 
    else: num_n += 1
    for j in range(NUM_FEATURES):
        if fp[j]: 
            feature_occ[j] += 1
            if tox == 1: num_p_s[j] += 1 
            else: num_n_s[j] += 1

In [68]:
np.set_printoptions(threshold=sys.maxsize)

weight = [0] * NUM_FEATURES
THRESHOLD = 0.3
MULTIPLIER = 10
for i in range(NUM_FEATURES):
    if feature_occ[i] == 0:
        weight[i] = 0
    else:
        weight[i] = log2((num_p_s[i] + 1) / (num_n_s[i] + 1))
        if abs(weight[i]) > THRESHOLD:
            weight[i] *= MULTIPLIER
        else:
            weight[i] = 1
weight = np.array(weight)
print(torch.Tensor(weight))

In [69]:
import numpy as np
from sklearn import svm
from datautil import *

train_set, test_set = read_data_as_np(None)
weighted_train_set, weighted_test_set = read_data_as_np(weight)

print(f"Loaded data : Matrix size {len(train_set)} x {len(train_set[0][0])}")

Loaded data : Matrix size 828 x 3501


In [70]:
def X_Y_Split(data_points):
    X, Y = [], []
    for idx in range(len(data_points)):
        x, y = data_points[idx]
        X.append(x)
        Y.append(y)
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

def evaluate_classifier(classifier, test_data, title=""):
    print(f"Evaluate {title}")
    testX, testY = X_Y_Split(test_data)
    total = testY.shape[0]
    testY = testY.ravel()
    pred = classifier.predict(testX)
    result = (pred==testY)
    correct = (np.count_nonzero(result))
    print(f"{correct}/{total} correct ({100*correct/total:.3f}%)")

tX, tY = X_Y_Split(train_set)
wtX, wtY = X_Y_Split(weighted_train_set)

In [71]:
from sklearn.svm import SVC
from sklearn.metrics.pairwise import *
from e3fp.fingerprint.metrics.array_metrics import soergel, tanimoto, dice, cosine, pearson

poly_clf = SVC(kernel='poly', degree=5, probability=False, coef0=0)
poly_clf.fit(tX, tY)
evaluate_classifier(poly_clf, test_set, "Degree 5 polynomial")

tanimoto_clf = SVC(kernel=tanimoto)
tanimoto_clf.fit(tX, tY)
evaluate_classifier(tanimoto_clf, test_set, "e3fp-tanimoto")

poly_clf = SVC(kernel='poly', degree=5, probability=False, coef0=0)
poly_clf.fit(wtX, wtY)
evaluate_classifier(poly_clf, test_set, "Degree 5 polynomial-w")

tanimoto_clf = SVC(kernel=tanimoto)
tanimoto_clf.fit(wtX, wtY)
evaluate_classifier(tanimoto_clf, test_set, "e3fp-tanimoto-w")

/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Evaluate Degree 5 polynomial
126/207 correct (60.870%)
Evaluate e3fp-tanimoto
142/207 correct (68.599%)


/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Evaluate Degree 5 polynomial-w
120/207 correct (57.971%)
Evaluate e3fp-tanimoto-w
123/207 correct (59.420%)


/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
